In [40]:
import re
import itertools
import collections
import numpy as np
import pandas as pd
import pickle
import brightway2 as bw
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

In [52]:
scores = pd.read_pickle('intermediate_res/lcia_res.p')

In [3]:
rel_scores_df = pd.read_pickle('intermediate_res/rel_scores_df.p')

In [4]:
scores.head()

aware_glo  aware_reg      ipcc  \
AT 0000afec84171c872e995c5b53fac1ad   5.325694   1.855518  4.467347   
   0002339e980aff56eb28e301ffac6e66  -0.007421  -0.000618 -0.005385   
   0002fe8e13e5d8537ea74a3a7e50f975   0.091101   0.013638  0.241638   
   0003a1862039cf71304a37965e1eac79   0.004159   0.000326  0.005495   
   00060f09c6a1a4289f685864b57c7163   3.343944   0.148095  1.080049   

                                       pslglo_glo    pslglo_reg    recipe  \
AT 0000afec84171c872e995c5b53fac1ad  6.716004e-16  4.396450e-16  0.676951   
   0002339e980aff56eb28e301ffac6e66 -9.988120e-18 -1.402385e-17 -0.000716   
   0002fe8e13e5d8537ea74a3a7e50f975  6.724522e-17  5.116796e-17  0.018839   
   0003a1862039cf71304a37965e1eac79  2.477572e-18  3.520357e-18  0.000941   
   00060f09c6a1a4289f685864b57c7163  2.131532e-16  1.329111e-16  0.118277   

                                     aware_eco  ipcc_eco    pslglo_eco  \
AT 0000afec84171c872e995c5b53fac1ad   5.355019  5.206169  6.898520e-16   
   0002339e980aff56eb28e301ffac6e66  -0.007278 -0.005493 -9.598913e-18   
   0002fe8e13e5d8537ea74a3a7e50f975   0.084489  0.273628  6.337074e-17   
   0003a1862039cf71304a37965e1eac79   0.004066  0.005749  2.272093e-18   
   00060f09c6a1a4289f685864b57c7163   3.282472  1.217052  1.747081e-16   

                                     recipe_eco  
AT 0000afec84171c872e995c5b53fac1ad    0.753082  
   0002339e980aff56eb28e301ffac6e66   -0.000721  
   0002fe8e13e5d8537ea74a3a7e50f975    0.021867  
   0003a1862039cf71304a37965e1eac79    0.000964  
   00060f09c6a1a4289f685864b57c7163    0.133209

In [5]:
rel_scores_df.head()

aware_glo  aware_reg      ipcc    recipe  \
AT 0000afec84171c872e995c5b53fac1ad   0.994524   0.346501  0.858087  0.898908   
   0002339e980aff56eb28e301ffac6e66   1.019728   0.084906  0.980307  0.991997   
   0002fe8e13e5d8537ea74a3a7e50f975   1.078256   0.161420  0.883089  0.861501   
   0003a1862039cf71304a37965e1eac79   1.022898   0.080299  0.955786  0.975688   
   00060f09c6a1a4289f685864b57c7163   1.018727   0.045117  0.887431  0.887907   

                                     pslglo_reg  pslglo_glo  
AT 0000afec84171c872e995c5b53fac1ad    0.637303    0.973543  
   0002339e980aff56eb28e301ffac6e66    1.460983    1.040547  
   0002fe8e13e5d8537ea74a3a7e50f975    0.807438    1.061140  
   0003a1862039cf71304a37965e1eac79    1.549390    1.090436  
   00060f09c6a1a4289f685864b57c7163    0.760761    1.220053

In [6]:
cutoff = None
rel_scores = rel_scores_df.copy()
rel_scores[rel_scores.isnull()] = 1
rel_scores[rel_scores == np.inf] = 1
rel_scores[rel_scores == -np.inf] = 1
if cutoff:
    rel_scores[rel_scores > cutoff] = cutoff
    rel_scores[rel_scores < -cutoff] = -cutoff

In [7]:
ipcc = rel_scores['ipcc']

In [8]:
ipcc.nlargest(10)

WL  1f0dd2c8c9e497d5e4db45f21de54e5b    10.975666
US  91ef98352b762931e18685b748224f8c     7.164202
CA  91c2c0f5fba234fba4fbc80e5cf3815b     6.714853
    35c181665a82a1c4d5c431cf48007fba     6.590340
    1997aa18e0083201879ca0b8e7b37b93     5.921726
    e1771d84202f766e8afb2c7883f247dc     5.903133
CH  0faf761acb398bbf3ba501eb32104620     5.199522
ES  efd02f8f6004f3d8be9a15eda7c25b3f     5.165418
CH  3b8d74f9c1fc2a3ab207dccc556577ef     4.929636
    ae6bb4e0cdc7de599f6bb07aeb9a8708     4.790307
Name: ipcc, dtype: float64

In [9]:
ipcc.nsmallest(10)

CA  7d1ba6ebb4bf8c3124bce538282a7115   -20.309833
TR  7d1ba6ebb4bf8c3124bce538282a7115   -18.374750
CA  a46f4a3c5920a287edd9b8b4ab7e2c75   -14.191067
    e6c07ed6fc2a0e5e3e2ff60e5de29dca   -14.177598
    0fa0de1ef68aad40ac6825f405c4b0d0   -13.652696
    d0edfd7fe072f4a364f9855ab4e98539   -13.652696
    52e5c6db31f64e2fa47bbcbd0fe176c2   -13.652695
    67e53ed694e4e20e243b8a9781e3224e   -13.652695
    34475266bb20732990bbcb3c2ac77461   -13.119242
TR  a46f4a3c5920a287edd9b8b4ab7e2c75   -12.655354
Name: ipcc, dtype: float64

In [10]:
with open('regeco/aux_data/matching_dicts.pickle', 'rb') as i:
    matching_dicts = pickle.load(i)

In [11]:
matching_dicts.keys()

dict_keys(['exio_to_isic', 'country_name_dict', 'isic_to_exio', 'exio_name_dict'])

In [12]:
with open('full_reference_dataset_dict.p', 'rb') as i:
    ref_ds_dict = pickle.load(i)

In [13]:
bw.projects.set_current('regeco')
db = bw.Database('cutoff33_no_mg')

In [14]:
def extract_isic_code(dataset):
    cl = dataset['classifications']
    try:
        isic_info = [c[1] for c in cl if c[0].startswith('ISIC')][0] # only production mixes have no isic-info
    except IndexError:
        #print(dataset['name'])
        return '0'
    isic_code = isic_info.split(':')[0]
    isic_code = re.sub(r'\D','', isic_code) # remove added non-digit characters
    return isic_code

In [15]:
ds = db.random()

In [16]:
ds['classifications']

[('ISIC rev.4 ecoinvent',
  '2013:Manufacture of plastics and synthetic rubber in primary forms'),
 ('EcoSpold01Categories', 'chemicals/inorganics')]

In [17]:
extract_isic_code(ds)

'2013'

In [18]:
isic_to_exio = matching_dicts['isic_to_exio']

In [19]:
isic_to_exio['8219']

['i74']

In [20]:
exio_name_dict = matching_dicts['exio_name_dict']

In [21]:
reference_datasets = set(ref_ds_dict.values())
len(reference_datasets)

12503

In [22]:
%time db_loaded = db.load()

CPU times: user 15.7 s, sys: 2.22 s, total: 17.9 s
Wall time: 17.9 s


In [23]:
list(db_loaded.keys())[:5]

[('cutoff33_no_mg', '1b9002783d2a31a87c3afb3840e0a72f'),
 ('cutoff33_no_mg', '4df2f6a69be9a464ebc1c7f20de24347'),
 ('cutoff33_no_mg', 'fe14f780a517767a54fe371d9d2ee5cc'),
 ('cutoff33_no_mg', '58a85b6a31a5795258895a26b161922e'),
 ('cutoff33_no_mg', '06122c6e75fae8be2c16fe0586d8fa27')]

In [24]:
%time isic_dict = {ds:extract_isic_code(db_loaded.get(('cutoff33_no_mg', ds))) for ds in reference_datasets}

CPU times: user 72 ms, sys: 0 ns, total: 72 ms
Wall time: 70.7 ms


('ISIC rev.4 ecoinvent', '3900:Remediation activities and other waste management services')

In [25]:
for k, v in isic_dict.items():
    if v == '0':
        isic_dict[k] = '3900'

In [26]:
sector_dict = collections.defaultdict(set)
for k, v in isic_dict.items():
    keys = [ke for ke in isic_to_exio.keys() if ke.startswith(v)]
    sectors = set(itertools.chain.from_iterable([isic_to_exio[key] for key in keys]))
    sector_dict[k].update(sectors)

In [27]:
sector_code_dict = collections.defaultdict(list)
for k, v in sector_dict.items():
    key = ' '.join(sorted(v))
    sector_code_dict[key].append(k)

In [28]:
full_index = rel_scores_df.index.tolist()

In [29]:
len(full_index)

307403

In [30]:
ref_ds_dict_inv = collections.defaultdict(list)
for k, v in ref_ds_dict.items():
    ref_ds_dict_inv[v].append(k)

In [31]:
sector_dict_inv = collections.defaultdict(list)
for k, v in sector_dict.items():
    for sec in v:
        sector_dict_inv[sec].append(k)

In [32]:
len(ref_ds_dict)

307403

In [ ]:
for code, datasets in sector_dict_inv.items():
    name = exio_name_dict[code]
    dss = set(itertools.chain.from_iterable([ref_ds_dict_inv[d] for d in datasets]))
    ind = pd.Index([i for i in full_index if i in dss])
    fig, ax = plt.subplots(figsize=(12, 6))
    #df = rel_scores.xs(region, axis=0, level=0)
    for m in ('ipcc', 'recipe'):#rel_scores.columns:
        vals = rel_scores.loc[ind, m].values.copy()
        vals.sort()
        ax.plot(vals, label=m)
        ax.legend()
        ax.set_title(name)
    ax.hlines(1, *ax.get_xlim())

In [51]:
for code, datasets in sector_dict_inv.items():
    name = exio_name_dict[code]
    dss = set(itertools.chain.from_iterable([ref_ds_dict_inv[d] for d in datasets]))
    ind = pd.Index([i for i in full_index if i in dss])
    for m in ('ipcc', 'recipe'):#rel_scores.columns:
        scores = rel_scores.loc[ind, m].copy()
        scores = scores[scores<0]
        if scores.min() < 0:
            print(m, name)
            display(scores.nsmallest())
            for i, s in scores.nsmallest().iteritems():
                if s <0:
                    ds_code = ref_ds_dict[i]
                    ds = db.get(ds_code)
                    print(ds['name'], s)
            print()

ipcc Lead, zinc and tin production


CA  181ec63ce60228a926eb2cc626e2d007   -0.963258
    9a09e9bbd61c2550d0ad5749b9ff8f59   -0.963258
    9e3324428628df73f60013f29039ea32   -0.963258
Name: ipcc, dtype: float64

magnesium production, electrolysis -0.9632581841565986
magnesium production, electrolysis -0.9632581838749068
magnesium production, electrolysis -0.9632581775785457

recipe Lead, zinc and tin production


CA  9a09e9bbd61c2550d0ad5749b9ff8f59   -0.888037
    181ec63ce60228a926eb2cc626e2d007   -0.888037
    9e3324428628df73f60013f29039ea32   -0.888037
Name: recipe, dtype: float64

magnesium production, electrolysis -0.8880373822013493
magnesium production, electrolysis -0.8880373819785986
magnesium production, electrolysis -0.8880373752029121

ipcc Copper production


...

polyester-complexed starch biopolymer production -0.23582648273817503
market for polyester-complexed starch biopolymer -0.2174859046252708

ipcc Sea and coastal water transport


CA  8b80b7e9439d32f0aa2c1b839b0fce17   -1.346696
    3b459e4bb3fb627639b1f457e000f893   -1.346696
Name: ipcc, dtype: float64

transport, freight, sea, liquefied natural gas -1.3466956084714439
market for transport, freight, sea, liquefied natural gas -1.3466956084714437

recipe Sea and coastal water transport


CA  3b459e4bb3fb627639b1f457e000f893   -1.215006
    8b80b7e9439d32f0aa2c1b839b0fce17   -1.215006
Name: recipe, dtype: float64

market for transport, freight, sea, liquefied natural gas -1.215005675086731
transport, freight, sea, liquefied natural gas -1.215005675086731



In [70]:
ipcc = scores[['ipcc', 'ipcc_eco']]

In [83]:
sign_change = ipcc/ipcc.abs()
changed = sign_change.loc[(sign_change['ipcc'] != sign_change['ipcc_eco']).values].dropna()

In [86]:
ipcc.loc[changed.index]

ipcc   ipcc_eco
AT 1b86f8d2360c6d2f749b8dc52c61c417  -0.103414   0.067613
   db75eda6edb27b2572f7690112f60ddd  -0.103414   0.067433
AU 17760a9c901e98e20db30da45d668239  -0.210997   0.844315
   1b86f8d2360c6d2f749b8dc52c61c417  -0.080855   0.067613
   c74deafc10fd19cc2608289d656b3f07  -0.210997   0.795906
   d475f9302366f087f1f1fbaffc216a5d  -0.052405   0.886054
   db75eda6edb27b2572f7690112f60ddd  -0.080855   0.067433
BE 17760a9c901e98e20db30da45d668239  -0.358615   0.844315
   1b86f8d2360c6d2f749b8dc52c61c417  -0.119964   0.067613
   84a74c6e316cd16bc7dab303cc7ce122  -2.204664   8.272929
   c74deafc10fd19cc2608289d656b3f07  -0.358615   0.795906
   d31c5f81a6d136ee579a27d698501b3b  -2.252447   8.220970
   db75eda6edb27b2572f7690112f60ddd  -0.119964   0.067433
BG 0b96caffccc12b8f6d3d3a561a8d9ca0  -2.310650   0.799068
   0fa0de1ef68aad40ac6825f405c4b0d0  -2.946706   0.434677
   10e2595d51068af21a5160f1a8cb20e9  -0.008403   0.028470
   14ecc3d97a6669a3e4faadf01eb1d511  -0.026117   0.074343
   15c2a7bf08edd78236076bf3555f9a1c  -0.025460   0.074812
   17760a9c901e98e20db30da45d668239  -6.127286   0.844315
   1b86f8d2360c6d2f749b8dc52c61c417  -0.116105   0.067613
   2d5c8864d02a6110e33f885c2e0f1b0c  -0.009974   0.029126
   32bf8a4f3c5e32d04262fe6773168cd3  -0.185712   0.633653
   34475266bb20732990bbcb3c2ac77461  -3.219428   0.491674
   422042d8ac79d48adcdddca48a2479fa  -0.009016   0.031187
   4e7ed653dd09864a9acf1a4cd34f7afc  -3.842611   0.682695
   4ef3711fa4fe5760dd91c0e838db6a5d  -3.842611   0.683034
   52e5c6db31f64e2fa47bbcbd0fe176c2 -23.984813   3.538067
   5719cc3c57c7b5cc07f57fcab273c3af  -0.079422   0.231837
   671b51535e23074963008612d9f213de  -3.473145   1.170073
   67e53ed694e4e20e243b8a9781e3224e -23.984813   3.538067
...                                        ...        ...
WA c33b53dde99d48438551721df7bf5e6b  -0.765034   0.742783
   c74deafc10fd19cc2608289d656b3f07  -0.049675   0.795906
   d57d0bea687db36e48cc3ba7be3d88ff  -0.765034   0.742783
   db75eda6edb27b2572f7690112f60ddd  -0.050709   0.067433
WF 6cd80f816916cc41b74bacf98c5d2bbd  -0.330207  13.409667
   84a74c6e316cd16bc7dab303cc7ce122  -2.344792   8.272929
   a0a6609c9c9dcd3be5cd9f806536c261  -0.993249   0.571498
   a1d18fe4c1a0fa0edcb6d8957aeceaa1  -0.050825   0.029244
   b5fad0bdadff7b7802b02bb5c8014584  -0.017326   0.581806
   d31c5f81a6d136ee579a27d698501b3b  -2.394548   8.220970
WL 0a11021d5aaba49476893ca52c05a6d6  -0.140546   0.739003
   0b96caffccc12b8f6d3d3a561a8d9ca0  -3.334188   0.799068
   1a3f22d946ff2077452dce03e631a2c2  -0.097798   0.790962
   38cd27437f90eff4a2fe283bf163119c  -1.267548   0.551672
   71a8fac5f043d4800cfafb362755e5ae  -3.395247   0.742783
   860180360915da2a3756f37fd599029e  -1.108265   0.963016
   8e6667a97d845894a327887ce3379910  -3.352150   0.794687
   8f14b72ac465a70509a1e82d8c22f682  -1.224800   0.603632
   a0a6609c9c9dcd3be5cd9f806536c261  -0.991812   0.571498
   a1d18fe4c1a0fa0edcb6d8957aeceaa1  -0.050751   0.029244
   c33b53dde99d48438551721df7bf5e6b  -3.395247   0.742783
   d2a0bed05c8459540384d88b61ba04ab   0.549981  -0.136025
   d57d0bea687db36e48cc3ba7be3d88ff  -3.395247   0.742783
ZA 17760a9c901e98e20db30da45d668239  -0.528684   0.844315
   1b86f8d2360c6d2f749b8dc52c61c417  -0.113916   0.067613
   a0a6609c9c9dcd3be5cd9f806536c261  -0.991894   0.571498
   a1d18fe4c1a0fa0edcb6d8957aeceaa1  -0.050756   0.029244
   c74deafc10fd19cc2608289d656b3f07  -0.528684   0.795906
   d475f9302366f087f1f1fbaffc216a5d  -0.422758   0.886054
   db75eda6edb27b2572f7690112f60ddd  -0.113916   0.067433

[540 rows x 2 columns]

In [87]:
for i in changed.index:
    ds = db.get(ref_ds_dict[i])
    print(ds['location'], ds['name'])

GLO market for natural gas, burned in gas motor, for storage
RoW natural gas, burned in gas motor, for storage
RoW natural gas production, liquefied
GLO market for natural gas, burned in gas motor, for storage
GLO market for natural gas, liquefied
RoW evaporation of natural gas
RoW natural gas, burned in gas motor, for storage
RoW natural gas production, liquefied
GLO market for natural gas, burned in gas motor, for storage
GLO market for helium
GLO market for natural gas, liquefied
GLO helium purification
RoW natural gas, burned in gas motor, for storage
GLO market for butane
GLO natural gas liquids production
RoW heat and power co-generation, natural gas, 500kW electrical, lean burn
Europe without Switzerland heat production, natural gas, at boiler atmospheric non-modulating <100kW
Europe without Switzerland heat production, natural gas, at boiler fan burner non-modulating <100kW
RoW natural gas production, liquefied
GLO market for natural gas, burned in gas motor, for storage
Europe

RoW polyester-complexed starch biopolymer production
RoW trimethylamine production
GLO methanol to generic market for solvent for chromatography analysis
GLO market for dimethyl ether
GLO transport, freight, sea, liquefied natural gas
RoW ammonia production, steam reforming, liquid
GLO market for ethane
GLO market for methanol
GLO market for drying, natural gas
GLO market for forging, steel
CA-QC heat production, natural gas, at boiler condensing modulating >100kW
RoW heat and power co-generation, natural gas, 1MW electrical, lean burn
GLO market for dimethyl sulfate
RoW chipboard production, white lined
RoW magnesium production, electrolysis
GLO market for trimethyl borate
RoW methyl tert-butyl ether production
CA-QC drying of maize grain
RoW magnesium production, electrolysis
GLO market for heat, central or small-scale, natural gas, Jakobsberg
GLO market for ammonium thiocyanate
RoW heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical
RoW heat and p

GLO market for natural gas, from medium pressure network (0.1-1 bar), at service station
RoW natural gas, from medium pressure network (0.1-1 bar), at service station
GLO market for helium, crude
RoW heat and power co-generation, natural gas, 160kW electrical, Jakobsberg
RoW natural gas, from low pressure network (<0.1 bar), at service station
GLO natural gas liquids production
Europe without Switzerland heat production, natural gas, at boiler condensing modulating <100kW
GLO ethane extraction, from natural gas liquids
Europe without Switzerland heat and power co-generation, natural gas, 160kW electrical, lambda=1
RoW natural gas pressure reduction from high to low pressure
GLO market for helium
GLO market for ethane
GLO market for heat, central or small-scale, natural gas, Jakobsberg
RoW heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical
RoW heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical
Europe without Switz

RoW heat and power co-generation, natural gas, 160kW electrical, lambda=1
RoW heat and power co-generation, natural gas, 200kW electrical, lean burn
RoW market for natural gas, low pressure
RoW market for fluting medium
RoW heat and power co-generation, natural gas, 50kW electrical, lean burn
RoW treatment of recovered paper to fluting medium, wellenstoff
GLO market for natural gas, from medium pressure network (0.1-1 bar), at service station
RoW natural gas, from medium pressure network (0.1-1 bar), at service station
GLO market for helium, crude
RoW heat and power co-generation, natural gas, 160kW electrical, Jakobsberg
RoW heat and power co-generation, natural gas, 1MW electrical, lean burn
RoW natural gas, from low pressure network (<0.1 bar), at service station
GLO natural gas liquids production
RoW heat production, natural gas, at boiler condensing modulating <100kW
RoW market for linerboard
RoW heat and power co-generation, natural gas, 160kW electrical, lambda=1
RoW natural gas

In [79]:
ipcc.isna().sum()

ipcc        0
ipcc_eco    0
dtype: int64

In [76]:
np.nan == np.nan

False